In [19]:
import pandas as pd


## Energy Consumption for heating in EU

Data from:
https://energy.ec.europa.eu/topics/energy-efficiency/energy-efficient-buildings/eu-building-stock-observatory_en

Also has data on age of buildings, not easily used though


In [243]:

consumption = pd.DataFrame(pd.read_excel("./energy_consumption.xlsx"))
columns = consumption.iloc[3].dropna()[1:-2].tolist()
consumption = consumption.iloc[4567:4771]
columns = ["Category", "Country", "Unit"] + [str(i) for i in columns]
columns = [i if "." not in i else str(int(float(i))) for i in columns]

blocks = consumption.iloc[:,6]
block_starts = blocks[blocks == "EU"].index.tolist()

heating_consumption = []

for i, start in enumerate(block_starts[:-1]):
    block_data = consumption.loc[start:block_starts[i+1]]
    category = block_data.iloc[:,4].dropna()
    data = block_data.iloc[:,6:-2]
    data.columns = columns[1:]
    data["Category"] = category.values[0]
    data = data[columns]

    heating_consumption.append(data)

heating_consumption = pd.concat(heating_consumption)


In [244]:
# Checking that the data makes sense

total = heating_consumption[heating_consumption["Category"] == "Residential-Space heating"]
f_total = total[total["Country"] == "Finland"]
other = heating_consumption[heating_consumption["Category"] != "Residential-Space heating"]
f_other = other[other["Country"] == "Finland"]

print(f_total["2000"].values[0], f_other["2000"].sum())


2.8092 2.8092


In [247]:
year = "2016"

for country in set(heating_consumption["Country"]):
    print(f"---------- {country} ----------")
    country_data = heating_consumption[heating_consumption["Country"] == country]
    country_total = country_data[country_data["Category"] == "Residential-Space heating"][year].values[0]
    for source in country_data["Category"]:
        if source != "Residential-Space heating":
            from_source = country_data[country_data["Category"] == source][year].values[0]
            print(f"Fraction produced by {source}: {from_source / country_total}")


---------- Germany ----------
Fraction produced by Natural gas: 0.5276653317458598
Fraction produced by Petroleum products: 0.26989122199372223
Fraction produced by Solid fuel: 0.01765613161597575
Fraction produced by Electricity: 0.02525976837320056
Fraction produced by Derivated heat: 0.09635241909297543
Fraction produced by Renewables: 0.06317242125771187
---------- Sweden ----------
Fraction produced by Natural gas: 0.005943678849610544
Fraction produced by Petroleum products: 0.011048532055122828
Fraction produced by Solid fuel: nan
Fraction produced by Electricity: 0.29526662672258835
Fraction produced by Derivated heat: 0.47439185140802875
Fraction produced by Renewables: 0.21332534451767524
---------- Czech Republic ----------
Fraction produced by Natural gas: 0.2702529618956132
Fraction produced by Petroleum products: 0.004717685985697513
Fraction produced by Solid fuel: 0.15544882057850357
Fraction produced by Electricity: 0.06421176219447114
Fraction produced by Derivated he